In [2]:
import torchvision
import matplotlib.pyplot as plt
import torch
import numpy as np
import pandas as pd
import os

from torchvision import transforms
from tqdm import tqdm
from functools import partialmethod

tqdm.__init__ = partialmethod(tqdm.__init__, disable = True)

In [5]:
class DataLoader():
    def __init__(self, train_transforms, val_transforms, dir_train, dir_val, batch_size):
        
        self.train_transforms = train_transforms
        self.val_transforms = val_transforms

        self.dir_train = dir_train
        self.dir_val = dir_val
        self.batch_size = batch_size

        train_data = torchvision.datasets.ImageFolder(self.dir_train, self.train_transforms)
        val_data = torchvision.datasets.ImageFolder(self.dir_val, self.val_transforms)

        self.train_dataloader = torch.utils.data.DataLoader(
            train_data, batch_size = self.batch_size, shuffle = True, num_workers = self.batch_size)
        self.val_dataloader = torch.utils.data.DataLoader(
            val_data, batch_size = self.batch_size, shuffle = True, num_workers = self.batch_size)
        
def train_model(model, loss, optimizer, sheduler, epochs, train_dataloader, val_dataloader):

    results = {
        'train_loss_history': [],
        'train_accuracy_history': [],
        'val_loss_history': [],
        'val_accuracy_history': []
    }

    for epoch in range(epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_dataloader
                sheduler.step()
                model.train()
            else:
                dataloader = val_dataloader
                model.eval()

            running_loss = 0
            running_accuracy = 0

            for inputs, labels in tqdm(dataloader):
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim = 1)

                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()

                running_loss += loss_value.item()
                running_accuracy += (preds_class == labels.data).float().mean()

            epoch_loss = running_loss / len(dataloader)
            epoch_accuracy = running_accuracy / len(dataloader)

            if phase == 'train':
                results['train_accuracy_history'].append(epoch_accuracy)
                results['train_loss_history'].append(epoch_loss)
            else:
                results['val_accuracy_history'].append(epoch_accuracy)
                results['val_loss_history'].append(epoch_loss)

    print('model training has been completed')
    return results
        

### Пайплайн обучения моделей resnet18, resnet50, resnet152

In [ ]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dir_train = './data/processed_data/train'
dir_val = './data/processed_data/val'

DataLoader_resnet = DataLoader(train_transforms, val_transforms, dir_train, dir_val, 8)

results = {}

resnet_models = {
    'resnet18': torchvision.models.resnet18(pretrained = True),
    'resnet50': torchvision.models.resnet50(pretrained = True),
    'resnet152': torchvision.models.resnet152(pretrained = True)
}

for model in resnet_models:
    for param in resnet_models[model].parameters():
        param.requires_grad = False

    resnet_models[model].fc = torch.nn.Linear(resnet_models[model].fc.in_features, 11)

    loss = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(resnet_models[model].parameters(), lr = 1.0e-3)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)

    results[model] = train_model(resnet_models[model], loss, optimizer, scheduler, 100,
                                 DataLoader_resnet.train_dataloader, DataLoader_resnet.val_dataloader)

In [11]:
results_resnet18 = pd.DataFrame(results['resnet18'])
results_resnet50 = pd.DataFrame(results['resnet50'])
results_resnet152 = pd.DataFrame(results['resnet152'])

results_resnet18['val_accuracy_history'] = [i.item() for i in results_resnet18['val_accuracy_history'].to_list()]
results_resnet18['train_accuracy_history'] = [i.item() for i in results_resnet18['train_accuracy_history'].to_list()]
results_resnet50['val_accuracy_history'] = [i.item() for i in results_resnet50['val_accuracy_history'].to_list()]
results_resnet50['train_accuracy_history'] = [i.item() for i in results_resnet50['train_accuracy_history'].to_list()]
results_resnet152['val_accuracy_history'] = [i.item() for i in results_resnet152['val_accuracy_history'].to_list()]
results_resnet152['train_accuracy_history'] = [i.item() for i in results_resnet152['train_accuracy_history'].to_list()]

results_resnet18.to_csv('./models_results/resnet18.csv')
results_resnet50.to_csv('./models_results/resnet50.csv')
results_resnet152.to_csv('./models_results/resnet152.csv')

### Пайплайн обучения моделей resnet152

In [ ]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees = (0, 20)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dir_train = './data/processed_data/train'
dir_val = './data/processed_data/val'

DataLoader_resnet152 = DataLoader(train_transforms, val_transforms, dir_train, dir_val, 8)

resnet152_results = {}

resnet152_1 = torchvision.models.resnet152(pretrained = True)

# Шаг градиентного спуска = 0.001
for param in resnet152_1.parameters():
    param.requires_grad = False

resnet152_1.fc = torch.nn.Linear(resnet152_1.fc.in_features, 11)

loss = torch.nn.CrossEntropyLoss()
optimizer_1 = torch.optim.Adam(resnet152_1.parameters(), lr = 1.0e-3)
scheduler_1 = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)

resnet152_results['resnet152_1'] = train_model(resnet152_1, loss, optimizer_1, scheduler_1, 20, 
                                               DataLoader_resnet152.train_dataloader, DataLoader_resnet152.val_dataloader)

# Шаг градиентного спуска = 0.002
resnet152_2 = torchvision.models.resnet152(pretrained = True)

for param in resnet152_1.parameters():
    param.requires_grad = False

resnet152_2.fc = torch.nn.Linear(resnet152_2.fc.in_features, 11)

loss = torch.nn.CrossEntropyLoss()
optimizer_2 = torch.optim.Adam(resnet152_2.parameters(), lr = 2.0e-3)
scheduler_2 = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)

resnet152_results['resnet152_2'] = train_model(resnet152_2, loss, optimizer_2, scheduler_2, 20,
                                               DataLoader_resnet152.train_dataloader, DataLoader_resnet152.val_dataloader)

# Шаг изменения градиента = 10
resnet152_3 = torchvision.models.resnet152(pretrained = True)

for param in resnet152_1.parameters():
    param.requires_grad = False

resnet152_3.fc = torch.nn.Linear(resnet152_3.fc.in_features, 11)

loss = torch.nn.CrossEntropyLoss()
optimizer_3 = torch.optim.Adam(resnet152_3.parameters(), lr = 1.0e-3)
scheduler_3 = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.1)

resnet152_results['resnet152_3'] = train_model(resnet152_3, loss, optimizer_3, scheduler_3, 20,
                                               DataLoader_resnet152.train_dataloader, DataLoader_resnet152.val_dataloader)

d:\Python\Neurons\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Python\Neurons\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
d:\Python\Neurons\venv\Lib\site-packages\torch\optim\lr_scheduler.py:192: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. 

model training has been completed


KeyboardInterrupt: 

In [ ]:
results_resnet152_1 = pd.DataFrame(resnet152_results['resnet152_1'])
results_resnet152_2 = pd.DataFrame(resnet152_results['resnet152_2'])
results_resnet152_3 = pd.DataFrame(resnet152_results['resnet152_3'])

results_resnet152_1['val_accuracy_history'] = [i.item() for i in results_resnet152_1['val_accuracy_history'].to_list()]
results_resnet152_1['train_accuracy_history'] = [i.item() for i in results_resnet152_1['train_accuracy_history'].to_list()]
results_resnet152_2['val_accuracy_history'] = [i.item() for i in results_resnet152_2['val_accuracy_history'].to_list()]
results_resnet152_2['train_accuracy_history'] = [i.item() for i in results_resnet152_2['train_accuracy_history'].to_list()]
results_resnet152_3['val_accuracy_history'] = [i.item() for i in results_resnet152_3['val_accuracy_history'].to_list()]
results_resnet152_3['train_accuracy_history'] = [i.item() for i in results_resnet152_3['train_accuracy_history'].to_list()]

results_resnet152_1.to_csv('./models_results/resnet152_1.csv')
results_resnet152_2.to_csv('./models_results/resnet152_2.csv')
results_resnet152_3.to_csv('./models_results/resnet152_3.csv')